In [40]:
using JuMP
struct Bus
    idx:: Int64
    name:: String
    p:: Matrix{Float64}
    max:: Float64
end
x = [2 3;
     4 5]
NODE = [Bus(1, "lima", x, 4), Bus(1, "lima", x, 5)]  
N = 1:2

1:2

In [41]:
m = Model()
@variable(m, P[N])
@constraint(m, c[n in N], P[n] <= NODE[n].max)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c[1] : P[1] ≤ 4
 c[2] : P[2] ≤ 5

In [1]:
using CSV, DataFrames, NamedArrays

function to_Structs(structure::DataType, inputs_dir:: String, filename:: String):: Vector{structure}
    file_dir = joinpath(inputs_dir, filename)
    struct_names = fieldnames(structure)
    struct_types = fieldtypes(structure)
    df = CSV.read(file_dir, DataFrame; types=Dict(zip(struct_names, struct_types)))
    cols = Tuple(df[!, col] for col in names(df))
    V = structure.(cols...)    
    return V
end

to_Structs (generic function with 1 method)

In [15]:
struct Bus
    bus_id:: Int64
    name:: String
    kv:: Float64
    type:: String
    lat:: Float64
    lon:: Float64
    slack:: Bool
end


In [ ]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "buses.csv"

BUSES = to_Structs(Bus, inputs_dir, filename)
B = NamedArray(BUSES, (getfield.(BUSES, :name)))

2-element Vector{Bus}:
 Bus(1, "san_diego", 120.0, "Substation", -1.0, -1.0, true)
 Bus(2, "lima", 120.0, "Substation", -1.0, -1.0, false)

In [25]:
B[2].kv

120.0

In [8]:
struct Load
    bus_id:: String
    sc_id:: String
    t_id:: String
    load:: Float64
end
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"

LOADS = to_Structs(Load, inputs_dir, filename)
LOADS

6-element Vector{Load}:
 Load("sd", "low", "1", 4.0)
 Load("sd", "low", "2", 5.0)
 Load("sd", "low", "3", 6.0)
 Load("lima", "high", "1", 2.0)
 Load("lima", "high", "3", 4.0)
 Load("lima", "high", "2", 1.0)

In [52]:
S = LOADS

dims = [:bus_id, :sc_id, :t_id]
val = :load

vals_in_dim = Vector{Vector{String}}(undef, length(dims))

for (i, d) in enumerate(dims)
    vals_in_dim[i] = unique(getfield.(S, d))
end

arr = Array{Union{Missing, Float64}}(missing, length.(vals_in_dim)...)
arr = NamedArray(arr, vals_in_dim, dims)
for s in S
    arr[getfield.(Ref(s), dims)...] = getfield(s, val)
end
arr

2×2×3 Named Array{Union{Missing, Float64}, 3}

[:, :, t_id=1] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     4.0  missing
lima           │ missing      2.0

[:, :, t_id=2] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     5.0  missing
lima           │ missing      1.0

[:, :, t_id=3] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     6.0  missing
lima           │ missing      4.0

In [49]:
S = LOADS
getfield.(Ref(S[1]), dims)

3-element Vector{String}:
 "sd"
 "low"
 "1"

In [4]:
T = Load
names = fieldnames(T)
types = fieldtypes(T)
Dict(zip(names, types))

Dict{Symbol, DataType} with 4 entries:
  :t_id   => String
  :value  => Float64
  :bus_id => String
  :sc_id  => String

In [20]:
using CSV, DataFrames

file = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs/scenarios.csv"
df = CSV.read(file, DataFrame)
cols = Tuple(df[!, col] for col in names(df))
S = Scenario.(cols...)


2-element Vector{Scenario}:
 Scenario(1, "low", 0.6)
 Scenario(2, "high", 0.4)

In [19]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "scenarios.csv"
SX = to_vec_of_structs(Scenario, inputs_dir, filename)

2-element Vector{Scenario}:
 Scenario(1, "low", 0.6)
 Scenario(2, "high", 0.4)

In [50]:
struct Load
    bus_id:: Int64
    sc_id:: Int64
    t_id:: Int64
    value:: Float64
end
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"
L = to_vec_of_structs(Load, inputs_dir, filename)

In [84]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"
load = CSV.read(joinpath(inputs_dir, "loads.csv"),DataFrame, 
types=[Int64, Int64, Int64, Float64]);
load

to_Structs (generic function with 2 methods)

In [88]:
using AxisKeys
x = wrapdims(load, :value, :bus_id, :scenario, :t_id)

(:bus_id, :sc_id, :t_id, :load)

In [ ]:
using CSV, DataFrames, NamedArrays

inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"

struct Load
    bus_id:: String
    sc_id:: String
    t_id:: String
    load:: Float64
end


#V = to_Structs(Load, inputs_dir, filename)

MethodError: MethodError: objects of type NTuple{4, Symbol} are not callable
The object of type `NTuple{4, Symbol}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

In [65]:
using AxisKeys

# Your 2D matrix
data_matrix = [1 1 10;
               2 2 20;
               3 1 30;
               1 2 40;
               2 1 200;
               3 2 100]

# Extract indices and values
indices = data_matrix[:, 1:2]
values = data_matrix[:, 3]

# Determine unique keys for each dimension
key1_values = unique(indices[:, 1])
key2_values = unique(indices[:, 2])

# Reshape values and create AxisArray
x = wrapdims(reshape(values, length(key1_values), length(key2_values)), key1_values, key2_values)



3-element Vector{String}:
 "sd"
 "low"
 "1"

In [67]:
reshape(values, length(key1_values), length(key2_values))